# **08. AZURE GPT 데이터 사용 중 추가 작업**

In [29]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from haversine import haversine, haversine_vector
import ast

apartment = pd.read_csv("apartment_20230910.csv")

In [30]:
apartment[apartment['기초'].isin(['화성시', '수원시', '평택시', '과천시', '성남시', '광명시', '하남시'])]['아파트명'].nunique()

127

**기초 단위 공통 정보 한 컬럼으로**

In [31]:
for_city = apartment[['광역', '기초', '소매','음식', '교육', '장례식장', '보건의료', '총인구수', '시군구내_통근통학', '타시군구_통근통학']]
for_city = for_city.drop_duplicates()
for_city.rename(columns={'타시군구/시군구내': '타시군구 통근통학/시군구내 통근통학', '음식': '음식점', '교육': '교육기관'}, inplace=True)

In [32]:
def add_column_name_as_suffix(column, value):
    if column in ['소매', '음식점', '교육기관', '장례식장', '보건의료']:
        return f"{column} 개수: {value}"
    elif column in ['시군구내_통근통학', '타시군구_통근통학', '타시군구/시군구내']:
        return f"{column} 인구: {value}"
    else:
        return f"{column}: {value}" 

columns_to_apply = ['소매', '음식점', '교육기관', '장례식장', '보건의료', '총인구수', '시군구내_통근통학', '타시군구_통근통학']

for column in columns_to_apply:
    for_city[column] = for_city.apply(lambda row: add_column_name_as_suffix(column, row[column]), axis=1)


In [33]:
for_city['도시마다 공통되는 정보'] = for_city.apply(lambda row:', '.join(map(str, row[:])), axis=1)
for_city.head()

,광역,기초,소매,음식점,교육기관,장례식장,보건의료,총인구수,시군구내_통근통학,타시군구_통근통학,도시마다 공통되는 정보
0,서울,강동구,소매 개수: 12030,음식점 개수: 4934,교육기관 개수: 1344,장례식장 개수: 19,보건의료 개수: 332,총인구수: 458614,시군구내_통근통학 인구: 124337,타시군구_통근통학 인구: 90334,"서울, 강동구, 소매 개수: 12030, 음식점 개수: 4934, 교육기관 개수: ..."
16,서울,강남구,소매 개수: 23667,음식점 개수: 11907,교육기관 개수: 5085,장례식장 개수: 25,보건의료 개수: 1909,총인구수: 538872,시군구내_통근통학 인구: 187047,타시군구_통근통학 인구: 77253,"서울, 강남구, 소매 개수: 23667, 음식점 개수: 11907, 교육기관 개수:..."
27,경기,성남시,소매 개수: 25354,음식점 개수: 11569,교육기관 개수: 3454,장례식장 개수: 51,보건의료 개수: 747,총인구수: 924087,시군구내_통근통학 인구: 315860,타시군구_통근통학 인구: 72548,"경기, 성남시, 소매 개수: 25354, 음식점 개수: 11569, 교육기관 개수:..."
37,경기,수원시,소매 개수: 29887,음식점 개수: 15726,교육기관 개수: 3979,장례식장 개수: 41,보건의료 개수: 681,총인구수: 1191620,시군구내_통근통학 인구: 416275,타시군구_통근통학 인구: 191154,"경기, 수원시, 소매 개수: 29887, 음식점 개수: 15726, 교육기관 개수:..."
64,경기,광명시,소매 개수: 7402,음식점 개수: 3008,교육기관 개수: 950,장례식장 개수: 10,보건의료 개수: 139,총인구수: 287603,시군구내_통근통학 인구: 77382,타시군구_통근통학 인구: 25828,"경기, 광명시, 소매 개수: 7402, 음식점 개수: 3008, 교육기관 개수: 9..."


In [34]:
for_city = for_city[for_city['기초'].isin(['화성시', '평택시', '수원시', '과천시', '성남시', '광명시', '하남시'])]
city_info = for_city['도시마다 공통되는 정보']
city_info

27      경기, 성남시, 소매 개수: 25354, 음식점 개수: 11569, 교육기관 개수:...
37      경기, 수원시, 소매 개수: 29887, 음식점 개수: 15726, 교육기관 개수:...
64      경기, 광명시, 소매 개수: 7402, 음식점 개수: 3008, 교육기관 개수: 9...
211     경기, 평택시, 소매 개수: 14493, 음식점 개수: 9621, 교육기관 개수: ...
272     경기, 화성시, 소매 개수: 22381, 음식점 개수: 11766, 교육기관 개수:...
568     경기, 과천시, 소매 개수: 1526, 음식점 개수: 547, 교육기관 개수: 25...
2071    경기, 하남시, 소매 개수: 10684, 음식점 개수: 3466, 교육기관 개수: ...
Name: 도시마다 공통되는 정보, dtype: object

In [35]:
city_info.to_csv("city_information.csv", encoding='UTF-8', index=False)

**아파트별 타입 정보 한 컬럼으로**

In [36]:
apartment_price = pd.read_csv("apartment_price_filtered.csv")
apartment_price.drop(columns=['전용율', '현관구조', '평당_공급액'], axis=1, inplace=True)
apartment_price.rename({'공급액(만원)': '공급액'}, axis=1, inplace=True)

In [37]:
def add_column_name_as_suffix(column, value):
    if column in ['공급면적', '전용면적']:
        return f"{column}: {value}㎡"
    elif column in ['방수', '욕실수']:
        return f"{column}: {value}개"
    else:
        return f"{column}: {value}만원" 
    
columns_to_apply = ['공급면적', '전용면적', '방수', '욕실수', '공급액']

for column in columns_to_apply:
    apartment_price[column] = apartment_price.apply(lambda row: add_column_name_as_suffix(column, row[column]), axis=1)


In [38]:
apartment_price['per_type'] = apartment_price[['공급면적', '전용면적', '방수', '욕실수', '공급액']].values.tolist()
apartment_price.drop(columns= ['공급면적', '전용면적', '방수', '욕실수', '공급액'], axis=1, inplace=True)

In [39]:
apartment_price = apartment_price.groupby('아파트명')['per_type'].apply(list).reset_index()
apartment_price = apartment_price.rename(columns={'per_type': 'total_type'})

In [40]:
apartment_price.to_csv("price.csv", encoding='UTF-8', index=False)

#### **배정받는 초등학교와 초등학교와의 거리**

In [41]:
school = pd.read_csv("school_location_20230322.csv", encoding='cp949')
gyeonggi_elementry = school.loc[(school['학교급구분'] == '초등학교') & (school['시도교육청명'] == '경기도교육청')
                                &(school['교육지원청명'].isin(['경기도성남교육지원청', '경기도수원교육지원청', '경기도광명교육지원청', '경기도평택교육지원청',
       '경기도화성오산교육지원청', '경기도안양과천교육지원청', '경기도광주하남교육지원청']))].copy()
gyeonggi_elementry['elementary_location'] = tuple(zip(gyeonggi_elementry['위도'], gyeonggi_elementry['경도']))
gyeonggi_elementry.drop(columns=['학교ID', '설립일자', '운영상태', '시도교육청코드', '시도교육청명', '교육지원청코드', '교육지원청명', '생성일자', '설립형태',\
                                '변경일자', '위도', '경도', '데이터기준일자', '소재지지번주소',	'소재지도로명주소', '학교급구분', '본교분교구분'],
                        axis=1, inplace=True)
gyeonggi_elementry.head(3)

,학교명,elementary_location
133,광명광성초등학교,"(37.47829616, 126.8728636)"
160,광명남초등학교,"(37.47552515, 126.8538613)"
161,광명동초등학교,"(37.48346267, 126.8632584)"


In [42]:
original = pd.read_csv('청약홈_합본.csv', encoding='cp949').drop('Unnamed: 0', axis=1)
original = original.copy()
original_piece = original[['아파트명', '초등학교_학군정보']].drop_duplicates()
original_piece['아파트명'] = original_piece['아파트명'].str.replace(" ", "").drop_duplicates()

In [43]:
our_data=apartment[apartment['기초'].isin(['화성시', '수원시', '평택시', '과천시', '성남시', '광명시', '하남시'])].drop_duplicates()
new=pd.merge(our_data, original_piece, on='아파트명', how='left')

아래는 데이터 원본에서 초등학교 학군정보가 없던 것

In [44]:
new.loc[new['초등학교_학군정보'].isna()]['아파트명'].unique()

array(['산성역자이푸르지오(성남)', '매교역푸르지오SKVIEW(수원)', '수원센트럴아이파크자이', '화성봉담자이라젠느',
       'e편한세상평택라씨엘로(2-1BL)', 'e편한세상평택하이센트(4BL)', '힐스테이트봉담프라이드시티2BL',
       '과천지식정보타운린파밀리에', '화서역푸르지오브리시엘', '평택지제역동문굿모닝힐맘시티', '평택고덕디에트르리비에르',
       '힐스테이트고덕스카이시티', '힐스테이트고덕센트럴', '서광교파크스위첸', '더샵광교산퍼스트파크',
       '한화포레나수원장안', '화성봉담2지구B-1BL중흥S-클래스', '쌍용더플래티넘오목천역',
       '화성봉담2지구B-4BL중흥S-클래스', '화성봉담2지구C-1BL중흥S-클래스',
       '화성향남상신지구A1-1블록향남역한양수자인디에스티지', 'e편한세상동탄파크아너스A56블럭', '위례아너스포레',
       '힐스테이트광교중앙역퍼스트'], dtype=object)

In [45]:
new.loc[new['초등학교_학군정보'] == '0']['아파트명'].unique()

array(['평택화양휴먼빌퍼스트시티', '화성비봉지구B3블록예미지센트럴에듀', '화성비봉공공주택지구B2블록호반써밋',
       '화성동탄2제일풍경채퍼스티어', '평택청북세종헤르메스', '과천한양수자인', '영통푸르지오트레센츠(A1BL)',
       '영통푸르지오파인베르(A2BL)'], dtype=object)

In [46]:
update = [('산성역자이푸르지오(성남)', '성남북초등학교'),
          ('매교역푸르지오SKVIEW(수원)', '권선초등학교'),
          ('수원센트럴아이파크자이', '인계초등학교'),
          ('화성봉담자이라젠느', '와우초등학교'),
          ('e편한세상평택라씨엘로(2-1BL)', '안중초등학교'),
          ('e편한세상평택하이센트(4BL)', '안중초등학교'),
          ('힐스테이트봉담프라이드시티2BL', '봉담초등학교'),
          ('과천지식정보타운린파밀리에', '과천율목초등학교'),
          ('화서역푸르지오브리시엘', '송림초등학교'),
          ('평택지제역동문굿모닝힐맘시티', '평택새빛초등학교'),
          ('평택고덕디에트르리비에르', '고덕초등학교'),
          ('힐스테이트고덕스카이시티', '율포초등학교'),
          ('힐스테이트고덕센트럴', '율포초등학교'),
          ('서광교파크스위첸', '창용초등학교'),
          ('더샵광교산퍼스트파크', '영화초등학교'),
          ('한화포레나수원장안', '수일초등학교'),
          ('화성봉담2지구B-1BL중흥S-클래스', '봉담초등학교'),
          ('쌍용더플래티넘오목천역', '오현초등학교'),
          ('화성봉담2지구B-4BL중흥S-클래스', '수현초등학교'),
          ('화성봉담2지구C-1BL중흥S-클래스', '봉담초등학교'),
          ('화성향남상신지구A1-1블록향남역한양수자인디에스티지', '상신초등학교'),
          ('e편한세상동탄파크아너스A56블럭', '한율초등학교'),
          ('위례아너스포레', '위례숲초등학교'),
          ('힐스테이트광교중앙역퍼스트', '신풍초등학교'),
          ('수원아이파크시티10단지', '남수원초등학교'),
          ('평택화양휴먼빌퍼스트시티', '현덕초등학교'),
          ('화성비봉지구B3블록예미지센트럴에듀', '비봉초등학교'),
          ('화성비봉공공주택지구B2블록호반써밋', '비봉초등학교'),
          ('화성동탄2제일풍경채퍼스티어', '화성신동초등학교'),
          ('평택청북세종헤르메스', '평택청아초등학교'),
          ('과천한양수자인', '문원초등학교'),
          ('영통푸르지오트레센츠(A1BL)', '잠원초등학교'),
          ('영통푸르지오파인베르(A2BL)', '잠원초등학교')]

for apartment_name, elementary in update:
    new.loc[new['아파트명'] == apartment_name, '초등학교_학군정보'] = elementary

In [47]:
new_elementary = pd.merge(new, gyeonggi_elementry, left_on = '초등학교_학군정보', right_on='학교명', how='left')
new_elementary[new_elementary['elementary_location'].isna()]['초등학교_학군정보'].unique()

array(['글빛초등학교', '과천율목초등학교', '율포초등학교', '동탄17초', '동탄18초'], dtype=object)

In [48]:
def geocoding(address):
    geo_local = Nominatim(user_agent='South Korea')
    location = geo_local.geocode(address)
    if location is not None and location.latitude is not None and location.longitude is not None:
        geo = (location.latitude, location.longitude)
        return geo
    else:
        return (0, 0)

update = [('글빛초등학교', '경기도 수원시 영통구 망포동 92-40'),
          ('과천율목초등학교', '경기도 과천시 갈현동 248'),
          ('율포초등학교', '경기도 평택시 고덕동 2620-4'),
          ('동탄17초', '경기도 화성시 신동 205'),
          ('동탄18초', '경기도 화성시 신동 254')]

for elementary_name, address in update:
    new_elementary.loc[new_elementary['초등학교_학군정보'] == elementary_name, 'elementary_location'] = str(geocoding(address))

new_elementary[new_elementary['초등학교_학군정보'].isin(['글빛초등학교', '과천율목초등학교', '율포초등학교', '동탄17초', '동탄18초'])]['아파트명'].unique()

array(['반정아이파크캐슬5단지', '과천지식정보타운린파밀리에', '힐스테이트고덕스카이시티', '힐스테이트고덕센트럴',
       '영통롯데캐슬엘클래스1블록', '동탄A106어울림파밀리에', '동탄A107숨마데시앙',
       '동탄신도시금강펜테리움6차센트럴파크(A59블럭)', '동탄파크릭스A52BL', '동탄파크릭스A51-2BL',
       '동탄파크릭스A51-1BL', '반정아이파크캐슬4단지', '영통아이파크캐슬3단지', '동탄파크릭스A55BL'],
      dtype=object)

In [49]:
def parse_tuple_string(string):
    try:
        result = ast.literal_eval(string)
        if isinstance(result, tuple):
            return result
        else:
            return result
    except (SyntaxError, ValueError):
        return string 
new_elementary['elementary_location'] = new_elementary['elementary_location'].apply(parse_tuple_string)

In [50]:
new_elementary['apartment_location'] = tuple(zip(new_elementary['위도'], new_elementary['경도']))

update = [('e편한세상평택라씨엘로(2-1BL)', '경기도 평택시 안중읍 현화리'),
          ('e편한세상평택하이센트(4BL)', '경기도 평택시 안중읍 현화리 757'),
          ('반정아이파크캐슬5단지', '경기도 화성시 반정동 621-85'),
          ('지제역반도체밸리제일풍경채2블록', '경기도 평택시 장당동 614'),
          ('평택지제역자이', '경기도 평택시 세교동 378-1'),
          ('영통롯데캐슬엘클래스1블록', '경기도 수원시 영통구 망포동 805'),
          ('영통롯데캐슬엘클래스2블록', '경기도 수원시 영통구 망포동 810'),
          ('동탄신도시금강펜테리움6차센트럴파크(A59블럭)','경기도 화성시 신동 165'),
          ('화성태안3지구우미린센트포레B-2블럭', '경기도 화성시 안녕동 164'),
          ('화성태안3지구우미린센트포레B-1블럭', '경기도 화성시 안녕동 164-120'),
          ('위례아너스포레', '경기도 하남시 학암동 642'),
          ('이안모란센트럴파크', '경기도 성남시 중원구 하대원동 147-5')]

for apartment_name, address in update:
    new_elementary.loc[new_elementary['아파트명'] == apartment_name, 'apartment_location'] = str(geocoding(address))

In [51]:
new_elementary['apartment_location'] = new_elementary['apartment_location'].apply(parse_tuple_string)

In [52]:
new_elementary['to_elementary'] = new_elementary.apply(lambda row: haversine(row['apartment_location'], row['elementary_location']), axis=1)
new_elementary[new_elementary['to_elementary'] > 2]['아파트명'].unique()

array(['평택화양휴먼빌퍼스트시티', '평택청북세종헤르메스', '힐스테이트봉담프라이드시티2BL', '봉담자이프라이드시티',
       '힐스테이트평택화양', '지제역반도체밸리제일풍경채2블록', 'e편한세상동탄파크아너스A56블럭', '반월자이더파크'],
      dtype=object)

In [53]:
new_elementary.drop(columns=['학교명','elementary_location', 'apartment_location'], axis=1, inplace=True)

In [54]:
new_elementary.to_csv("plus_elementary.csv", encoding='UTF-8', index=False)